In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra


In [2]:
!pip install timm

In [3]:
import os
from glob import glob


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2

from sklearn.model_selection import train_test_split
from sklearn import metrics

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from tqdm.notebook import tqdm
import albumentations as A

import timm

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
data=pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')
data.head()

,image_name,points
0,medicakolkata_mau_mon--2_2022_6_8_6_8_51.jpeg,"[459.5, 169.5, 886.0999755859375, 172.19999694..."
1,kamalnayanbajaj_micu_mon--22_2022_6_20_17_47_2...,"[422.4, 158, 967.8, 135.5, 957.3, 545.2, 409.5..."
2,kamalnayanbajaj_micu_mon--15_2022_6_20_22_46_2...,"[497.4, 184.9, 792.7999877929688, 229.19999694..."
3,medicakolkata_ccu2_mon--3_2022_5_30_12_6_19.jpeg,"[353.79998779296875, 112.69999694824219, 764.2..."
4,medicakolkata_ccu2_mon--5_2022_6_8_6_6_37.jpeg,"[246.9, 155.3, 756.5999755859375, 205.89999389..."


In [5]:
def transform(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """
    Normalize an image tensor with the given mean and standard deviation.

    Args:
    - img (torch.Tensor): The image tensor to normalize, expected shape (C, H, W).
    - mean (list): The mean values for each channel.
    - std (list): The standard deviation values for each channel.

    Returns:
    - torch.Tensor: The normalized image tensor.
    """
    # Convert the mean and std to tensors
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    
    # Normalize the image
    img = (img - mean) / std
    
    return img


In [6]:
import json
class NaturalDataset(Dataset):

    # constructor
    def __init__(self, train = True,transform=None):

        super().__init__()
        train_df=pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')
        # initialize lists to store paths of images and their corresponding labels
        self.image_paths = []
        self.labels = []
        image_dir='/kaggle/input/iitgai-ovha-23/images'
        
        for i in range(train_df.shape[0]):
            self.image_paths.append(f"{image_dir}/{train_df.image_name[i]}")
            string_representation=train_df.points[i]
            extracted_list=json.loads(string_representation)
            numpy_array=np.array(extracted_list)
            self.labels.append(numpy_array)

       
        # train_test_split (random state has to be fixed to maintain the same split)
        image_train, image_val, label_train, label_val = train_test_split(self.image_paths, self.labels, shuffle=True, test_size = 0.1, random_state=2000)

        if train:
            self.image_paths = image_train
            self.labels = label_train
            if transform is not None:
                self.transform=transform
        else:
            self.image_paths = image_val
            self.labels = label_val
            if transform is not None:
                self.transform=transform

        # function for augumentation and preprocessing
        


    # len function
    def __len__(self):
        return len(self.image_paths)


    # function to return the pair (image, label)
    def __getitem__(self, idx):

        # read image
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        #img = img[:, :, ::-1]

        

        # making the required shape of the image
        img = img.transpose(2, 0, 1)

        # image tensor
        img_tensor = torch.tensor(img, dtype=torch.float)
        img_tensor=self.transform(img_tensor) ## normalization

        # label tensor
        label = self.labels[idx]
        label_tensor = torch.tensor(label, dtype=torch.float)

        return img_tensor, label_tensor

In [7]:
train_dataset = NaturalDataset(train=True,transform=transform)
train_loader = DataLoader(train_dataset,
                          batch_size = 8,
                          num_workers = 2,
                          drop_last = True,
                          shuffle = True)

In [8]:
val_dataset = NaturalDataset(train=False,transform=transform)
val_loader = DataLoader(val_dataset,
                          batch_size = 8,
                          num_workers = 2,
                          drop_last = False,
                          shuffle = False)

In [9]:
pretrained_model = timm.create_model('resnet18', pretrained = True, num_classes = 0)
pretrained_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [10]:
class CustomModel(nn.Module):

    def __init__(self, num_classes = 8):
        super().__init__()

        # initialize backbone using pretrained model (hint: use num_classes = 0)
        self.backbone = timm.create_model('resnet18', pretrained = True, num_classes = 0)

        # 2 linear layers after the backbone
        self.linear1 = nn.Linear(512,120)
        self.linear2 = nn.Linear(120,8)
        self.relu=nn.ReLU() 
        # softmax function
        

    def forward(self, x):
        x = self.backbone(x)
        

        x = self.linear1(x)
        
        x=self.relu(x)
        x = self.linear2(x)
        

        return x

In [11]:
model=CustomModel()

In [12]:
model.to(DEVICE)

CustomModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=

In [13]:
criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [14]:
def train_fn(train_loader, model, criterion, optimizer):

    # list to store the loss calculated on each batch
    losses = []

    # to enable batchnorm, dropout etc.
    model.train()

    progress = tqdm(train_loader, total=len(train_loader))

    for i, (imgs, labels) in enumerate(progress):

        # image and label tensors should be on the same device
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Propogation and calculation of loss
        y_preds = model(imgs)
        loss = criterion(y_preds, labels)

        # Back Propogation
        optimizer.zero_grad()         # Clearing all previous gradients, setting them to zero
        loss.backward()               # Calculating the new gradients (backprop)
        optimizer.step()              # Updating the weights using the gradients

        # appending the loss to the list
        losses.append(loss.item())

    # returning the mean loss overhe epoch
    return  np.mean(losses)

In [15]:
def val_fn(val_loader, model, criterion, optimizer):

    # list to store the loss calculated on each batch
    losses = []

    # to disable batchnorm, dropout etc.
    model.eval()

    progress = tqdm(val_loader, total=len(val_loader))

    for i, (imgs, labels) in enumerate(progress):

        # image and label tensors should be on the same device as that of model
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        # to disable the calculation of gradients
        with torch.no_grad():

            # Forward Propogation and calculation of loss
            y_preds = model(imgs)
            loss = criterion(y_preds,labels)

        # appending the loss to the list
        losses.append(loss.item())

    # returning the mean loss over the epoch
    return np.mean(losses)

In [17]:
EPOCHS = 16

# list to store the loss of each epoch
train_losses = []
val_losses = []

# for storing the best weights of the model
best_dict = None
best_loss = np.inf

for ep in range(EPOCHS):

    print('='*5 + f" Epoch {ep+1} " + '='*5)

    # training and validation loop
    tr_loss = train_fn(train_loader, model, criterion, optimizer)
    val_loss = val_fn(val_loader, model, criterion, optimizer)

    # updating the best weights, if loss is even less than the best one
    if val_loss < best_loss:
        best_loss = val_loss
        best_dict = model.state_dict()

    # appending the loss of each epoch
    train_losses.append(tr_loss)
    val_losses.append(val_loss)

    print(f"Epoch {ep} - Train Loss {tr_loss:.4f} - Val Loss {val_loss:.4f}\n")

===== Epoch 1 =====


  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 0 - Train Loss 48586.5032 - Val Loss 7771.6541

===== Epoch 2 =====


  0%|          | 0/202 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 14 - Train Loss 3017.3075 - Val Loss 2610.2836

===== Epoch 16 =====


  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 15 - Train Loss 2728.2325 - Val Loss 2618.9449



In [18]:
model.load_state_dict(best_dict)

<All keys matched successfully>

In [21]:
EPOCHS = 30

# list to store the loss of each epoch
train_losses = []
val_losses = []


for ep in range(EPOCHS):

    print('='*5 + f" Epoch {ep+1} " + '='*5)

    # training and validation loop
    tr_loss = train_fn(train_loader, model, criterion, optimizer)
    val_loss = val_fn(val_loader, model, criterion, optimizer)

    # updating the best weights, if loss is even less than the best one
    if val_loss < best_loss:
        best_loss = val_loss
        best_dict = model.state_dict()

    # appending the loss of each epoch
    train_losses.append(tr_loss)
    val_losses.append(val_loss)

    print(f"Epoch {ep} - Train Loss {tr_loss:.4f} - Val Loss {val_loss:.4f}\n")

===== Epoch 1 =====


  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 0 - Train Loss 1482.4029 - Val Loss 1387.3430

===== Epoch 2 =====


  0%|          | 0/202 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 28 - Train Loss 257.0618 - Val Loss 455.3676

===== Epoch 30 =====


  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 29 - Train Loss 259.3361 - Val Loss 275.9611



In [22]:
model.load_state_dict(best_dict)

<All keys matched successfully>

In [23]:
# list to store predicted and the true labels
val_preds = []
val_trues = []

# prediction on validation dataset
progress = tqdm(val_loader, total=len(val_loader))

# to disable batchnorm, dropout etc.
model.eval()

for i, (imgs, labels) in enumerate(progress):

    # image and label tensors should be on the same device as that of model
    imgs = imgs.to(DEVICE)
    labels = labels.to(DEVICE)

    # forward propogation
    with torch.no_grad():
        y_preds = model(imgs)

   
     

    # appending the predicted and true labels to the corresponding lists
    val_preds.extend(y_preds.tolist())
    val_trues.extend(labels.cpu().numpy().tolist())

  0%|          | 0/23 [00:00<?, ?it/s]

In [24]:
val_preds[0]

[478.0431823730469,
 308.7445983886719,
 726.567626953125,
 299.4227600097656,
 718.6004638671875,
 501.7144470214844,
 471.03802490234375,
 514.103271484375]

In [25]:
val_trues[0]

[436.6000061035156,
 309.8999938964844,
 696.7999877929688,
 302.1000061035156,
 698.0,
 501.1000061035156,
 435.5,
 516.2000122070312]

In [ ]:
import json
class TestDataset(Dataset):

    # constructor
    def __init__(self, train = True):

        super().__init__()
        train_df=pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')
        # initialize lists to store paths of images and their corresponding labels
        self.image_paths = []
        self.labels = []
        image_dir='/kaggle/input/iitgai-ovha-23/test_images'
        
        for i in range(train_df.shape[0]):
            self.image_paths.append(f"{image_dir}/{train_df.image_name[i]}")
            
       
        


    # len function
    def __len__(self):
        return len(self.image_paths)


    # function to return the pair (image, label)
    def __getitem__(self, idx):

        # read image
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        #img = img[:, :, ::-1]

        

        # making the required shape of the image
        img = img.transpose(2, 0, 1)

        # image tensor
        img_tensor = torch.tensor(img, dtype=torch.float)
        
        
        

        return img_tensor

In [ ]:
test_dataset = TestDataset(train=True)
test_loader = DataLoader(test_dataset,
                          batch_size = 8,
                          num_workers = 2,
                          drop_last = True,
                          shuffle = True)

In [ ]:
test_preds = []


# prediction on validation dataset
progress = tqdm(test_loader, total=len(test_loader))

# to disable batchnorm, dropout etc.
model.eval()

for i, imgs in enumerate(progress):

    # image and label tensors should be on the same device as that of model
    imgs = imgs.to(DEVICE)
    

    # forward propogation
    with torch.no_grad():
        y_preds = model(imgs)

   
     

    # appending the predicted and true labels to the corresponding lists
    test_preds.extend(f'{y_preds.tolist()}')
    

In [ ]:
data=pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')
data.head()

In [ ]:
data=data.drop(columns='points')


In [ ]:
data['points']=pd.Series(test_preds)

In [ ]:
data.head()

In [ ]:
data.to_csv('/kaggle/working/submission.csv')